<a href="https://colab.research.google.com/github/devmatsuko/PBL-06/blob/main/PBL06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PBL06_データマーケティング

## ライブラリのインポート

In [48]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

## データの読み込み

In [49]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/train.csv')
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/test.csv")
coupon = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/coupon.csv')
customer = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/customer.csv")
purchase = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/purchase_history.csv")

## 読み込んだデータの詳細を表示

In [50]:
train.info()
train

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3916 entries, 0 to 3915
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   顧客ID    3916 non-null   int64
 1   クーポンID  3916 non-null   int64
 2   クーポン利用  3916 non-null   int64
dtypes: int64(3)
memory usage: 91.9 KB


,顧客ID,クーポンID,クーポン利用
0,1,1,0
1,1,2,0
2,1,3,0
3,1,4,0
4,1,5,0
...,...,...,...
3911,356,7,0
3912,356,8,0
3913,356,9,0
3914,356,10,0


In [51]:
test.info()
test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3916 entries, 0 to 3915
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   顧客ID    3916 non-null   int64
 1   クーポンID  3916 non-null   int64
dtypes: int64(2)
memory usage: 61.3 KB


,顧客ID,クーポンID
0,357,1
1,357,2
2,357,3
3,357,4
4,357,5
...,...,...
3911,712,7
3912,712,8
3913,712,9
3914,712,10


In [52]:
# クーポン情報（クーポンIDとクーポンを使用できる商品カテゴリの対応）を表示
coupon.info()
coupon

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   クーポンID    11 non-null     int64 
 1   商品カテゴリID  11 non-null     object
 2   商品カテゴリ    11 non-null     object
dtypes: int64(1), object(2)
memory usage: 392.0+ bytes


,クーポンID,商品カテゴリID,商品カテゴリ
0,1,c1,米・パン
1,2,c2,肉
2,3,c3,海鮮
3,4,c4,自然食品
4,5,c5,即席食品
5,6,c6,惣菜
6,7,c7,飲料
7,8,c8,食料品その他
8,9,c9,医薬品類
9,10,c10,スキンケア・ヘアケア


In [53]:
# 顧客の属性情報を表示
customer.info()
customer

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   顧客ID    712 non-null    int64 
 1   年齢層     712 non-null    object
 2   既婚・独身   377 non-null    object
 3   家族人数    712 non-null    object
 4   子供人数    184 non-null    object
 5   持家      712 non-null    int64 
 6   年収レベル   712 non-null    int64 
dtypes: int64(3), object(4)
memory usage: 39.1+ KB


,顧客ID,年齢層,既婚・独身,家族人数,子供人数,持家,年収レベル
0,1,40-49,NaN,1,NaN,0,5
1,2,70+,NaN,1,NaN,0,5
2,3,40-49,独身,5+,3+,0,2
3,4,50-59,既婚,5+,3+,0,9
4,5,20-29,NaN,2,NaN,0,4
...,...,...,...,...,...,...,...
707,708,60-69,既婚,2,NaN,0,2
708,709,60-69,既婚,2,NaN,0,5
709,710,50-59,NaN,2,NaN,0,6
710,711,50-59,NaN,1,NaN,0,10


In [54]:
# 各顧客の直近2ヶ月の購買履歴を表示 
# ※c1=クーポン1, c2=クーポン2, ... c11=クーポン11を表し、クーポンの商品カテゴリ別の購入回数、購入商品数、クーポン割引金額、購入金額の情報が含まれる
# ※「経過日数」は前回の来店から経過した日数を表す
purchase.info()
purchase

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 48 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   顧客ID        712 non-null    int64
 1   c1_購入回数     712 non-null    int64
 2   c1_購入商品数    712 non-null    int64
 3   c1_クーポン割引   712 non-null    int64
 4   c1_購入金額     712 non-null    int64
 5   c2_購入回数     712 non-null    int64
 6   c2_購入商品数    712 non-null    int64
 7   c2_クーポン割引   712 non-null    int64
 8   c2_購入金額     712 non-null    int64
 9   c3_購入回数     712 non-null    int64
 10  c3_購入商品数    712 non-null    int64
 11  c3_クーポン割引   712 non-null    int64
 12  c3_購入金額     712 non-null    int64
 13  c4_購入回数     712 non-null    int64
 14  c4_購入商品数    712 non-null    int64
 15  c4_クーポン割引   712 non-null    int64
 16  c4_購入金額     712 non-null    int64
 17  c5_購入回数     712 non-null    int64
 18  c5_購入商品数    712 non-null    int64
 19  c5_クーポン割引   712 non-null    int64
 20  c5_購入金額     712 non-null    int6

,顧客ID,c1_購入回数,c1_購入商品数,c1_クーポン割引,c1_購入金額,c2_購入回数,c2_購入商品数,c2_クーポン割引,c2_購入金額,c3_購入回数,c3_購入商品数,c3_クーポン割引,c3_購入金額,c4_購入回数,c4_購入商品数,c4_クーポン割引,c4_購入金額,c5_購入回数,c5_購入商品数,c5_クーポン割引,c5_購入金額,c6_購入回数,c6_購入商品数,c6_クーポン割引,c6_購入金額,c7_購入回数,c7_購入商品数,c7_クーポン割引,c7_購入金額,c8_購入回数,c8_購入商品数,c8_クーポン割引,c8_購入金額,c9_購入回数,c9_購入商品数,c9_クーポン割引,c9_購入金額,c10_購入回数,c10_購入商品数,c10_クーポン割引,c10_購入金額,c11_購入回数,c11_購入商品数,c11_クーポン割引,c11_購入金額,来店頻度,経過日数,総購入金額
0,1,1,2,0,534,1,2,0,2803,0,0,0,0,1,1,0,764,9,17,0,4981,0,0,0,0,0,0,0,0,9,77,0,22620,9,10,0,24860,0,0,0,0,0,0,0,0,9,19,56562
1,2,0,0,0,0,10,10,0,4278,4,5,0,2311,5,18,0,1104,8,9,-178,5423,0,0,0,0,8,13,0,3866,13,87,0,14527,13,18,0,5804,0,0,0,0,0,0,0,0,13,6,37313
2,3,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,407,0,0,0,0,0,0,0,0,0,0,0,0,13,26,0,7986,13,39,-489,38665,0,0,0,0,0,0,0,0,13,8,47058
3,4,2,3,0,886,2,2,0,1684,2,2,0,2133,9,9,0,4616,13,19,0,9471,7,8,0,3921,3,3,0,1576,14,150,-445,73174,14,40,0,24308,1,1,0,746,0,0,0,0,14,27,122515
4,5,1,1,0,267,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,712,6,40,-178,18372,3,3,-267,3380,2,2,0,1519,0,0,0,0,6,48,24250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,708,1,1,0,354,0,0,0,0,1,1,0,1066,7,8,0,3360,2,2,0,1688,1,1,0,710,1,1,0,443,14,59,0,29945,5,6,0,3198,0,0,0,0,4,4,0,5464,14,7,46228
708,709,15,15,0,5519,2,2,0,1401,0,0,0,0,9,17,0,3875,3,3,0,1900,3,3,0,2716,22,33,0,15982,22,116,0,46681,9,13,0,5095,0,0,0,0,2,2,0,2669,22,3,85838
709,710,0,0,0,0,0,0,0,0,0,0,0,0,1,3,0,1022,0,0,0,0,0,0,0,0,1,1,0,297,1,3,0,528,1,2,0,1367,0,0,0,0,0,0,0,0,1,58,3214
710,711,11,16,0,2935,2,4,0,1544,5,7,0,3670,5,8,0,2844,27,36,-267,15886,9,9,0,4578,10,24,0,3451,50,585,-1754,184915,50,108,-178,47135,1,1,0,1155,2,2,0,3025,50,1,271138


## train,testデータと他データの結合

### trainと顧客属性を結合(顧客ID、クーポンIDをキーに、クーポンごとの使用回数と顧客情報が載る)

In [55]:
# trainと顧客属性を結合(顧客ID、クーポンIDをキーに、クーポンごとの使用回数と顧客情報が載る)
merge_train = pd.merge(train, customer, on='顧客ID', how='left')
merge_train

,顧客ID,クーポンID,クーポン利用,年齢層,既婚・独身,家族人数,子供人数,持家,年収レベル
0,1,1,0,40-49,NaN,1,NaN,0,5
1,1,2,0,40-49,NaN,1,NaN,0,5
2,1,3,0,40-49,NaN,1,NaN,0,5
3,1,4,0,40-49,NaN,1,NaN,0,5
4,1,5,0,40-49,NaN,1,NaN,0,5
...,...,...,...,...,...,...,...,...,...
3911,356,7,0,50-59,独身,2,NaN,0,4
3912,356,8,0,50-59,独身,2,NaN,0,4
3913,356,9,0,50-59,独身,2,NaN,0,4
3914,356,10,0,50-59,独身,2,NaN,0,4


### trainと顧客属性を結合した物に、さらに購買履歴を結合する

In [56]:
# trainと顧客属性を結合した物に、さらに購買履歴を結合する
merge_train = pd.merge(merge_train, purchase, on='顧客ID', how='left')
merge_train

,顧客ID,クーポンID,クーポン利用,年齢層,既婚・独身,家族人数,子供人数,持家,年収レベル,c1_購入回数,c1_購入商品数,c1_クーポン割引,c1_購入金額,c2_購入回数,c2_購入商品数,c2_クーポン割引,c2_購入金額,c3_購入回数,c3_購入商品数,c3_クーポン割引,c3_購入金額,c4_購入回数,c4_購入商品数,c4_クーポン割引,c4_購入金額,c5_購入回数,c5_購入商品数,c5_クーポン割引,c5_購入金額,c6_購入回数,c6_購入商品数,c6_クーポン割引,c6_購入金額,c7_購入回数,c7_購入商品数,c7_クーポン割引,c7_購入金額,c8_購入回数,c8_購入商品数,c8_クーポン割引,c8_購入金額,c9_購入回数,c9_購入商品数,c9_クーポン割引,c9_購入金額,c10_購入回数,c10_購入商品数,c10_クーポン割引,c10_購入金額,c11_購入回数,c11_購入商品数,c11_クーポン割引,c11_購入金額,来店頻度,経過日数,総購入金額
0,1,1,0,40-49,NaN,1,NaN,0,5,1,2,0,534,1,2,0,2803,0,0,0,0,1,1,0,764,9,17,0,4981,0,0,0,0,0,0,0,0,9,77,0,22620,9,10,0,24860,0,0,0,0,0,0,0,0,9,19,56562
1,1,2,0,40-49,NaN,1,NaN,0,5,1,2,0,534,1,2,0,2803,0,0,0,0,1,1,0,764,9,17,0,4981,0,0,0,0,0,0,0,0,9,77,0,22620,9,10,0,24860,0,0,0,0,0,0,0,0,9,19,56562
2,1,3,0,40-49,NaN,1,NaN,0,5,1,2,0,534,1,2,0,2803,0,0,0,0,1,1,0,764,9,17,0,4981,0,0,0,0,0,0,0,0,9,77,0,22620,9,10,0,24860,0,0,0,0,0,0,0,0,9,19,56562
3,1,4,0,40-49,NaN,1,NaN,0,5,1,2,0,534,1,2,0,2803,0,0,0,0,1,1,0,764,9,17,0,4981,0,0,0,0,0,0,0,0,9,77,0,22620,9,10,0,24860,0,0,0,0,0,0,0,0,9,19,56562
4,1,5,0,40-49,NaN,1,NaN,0,5,1,2,0,534,1,2,0,2803,0,0,0,0,1,1,0,764,9,17,0,4981,0,0,0,0,0,0,0,0,9,77,0,22620,9,10,0,24860,0,0,0,0,0,0,0,0,9,19,56562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3911,356,7,0,50-59,独身,2,NaN,0,4,1,2,0,178,0,0,0,0,0,0,0,0,0,0,0,0,3,3,0,1512,0,0,0,0,2,2,0,1100,6,16,0,6114,2,3,0,1011,1,1,0,657,0,0,0,0,6,12,10572
3912,356,8,0,50-59,独身,2,NaN,0,4,1,2,0,178,0,0,0,0,0,0,0,0,0,0,0,0,3,3,0,1512,0,0,0,0,2,2,0,1100,6,16,0,6114,2,3,0,1011,1,1,0,657,0,0,0,0,6,12,10572
3913,356,9,0,50-59,独身,2,NaN,0,4,1,2,0,178,0,0,0,0,0,0,0,0,0,0,0,0,3,3,0,1512,0,0,0,0,2,2,0,1100,6,16,0,6114,2,3,0,1011,1,1,0,657,0,0,0,0,6,12,10572
3914,356,10,0,50-59,独身,2,NaN,0,4,1,2,0,178,0,0,0,0,0,0,0,0,0,0,0,0,3,3,0,1512,0,0,0,0,2,2,0,1100,6,16,0,6114,2,3,0,1011,1,1,0,657,0,0,0,0,6,12,10572


### testと顧客情報と購買履歴を結合する

In [57]:
# testと顧客属性, 購買履歴データを結合
merge_test = pd.merge(test, customer, on='顧客ID', how='left')
merge_test = pd.merge(merge_test, purchase, on='顧客ID', how='left')

In [58]:
merge_test

,顧客ID,クーポンID,年齢層,既婚・独身,家族人数,子供人数,持家,年収レベル,c1_購入回数,c1_購入商品数,c1_クーポン割引,c1_購入金額,c2_購入回数,c2_購入商品数,c2_クーポン割引,c2_購入金額,c3_購入回数,c3_購入商品数,c3_クーポン割引,c3_購入金額,c4_購入回数,c4_購入商品数,c4_クーポン割引,c4_購入金額,c5_購入回数,c5_購入商品数,c5_クーポン割引,c5_購入金額,c6_購入回数,c6_購入商品数,c6_クーポン割引,c6_購入金額,c7_購入回数,c7_購入商品数,c7_クーポン割引,c7_購入金額,c8_購入回数,c8_購入商品数,c8_クーポン割引,c8_購入金額,c9_購入回数,c9_購入商品数,c9_クーポン割引,c9_購入金額,c10_購入回数,c10_購入商品数,c10_クーポン割引,c10_購入金額,c11_購入回数,c11_購入商品数,c11_クーポン割引,c11_購入金額,来店頻度,経過日数,総購入金額
0,357,1,70+,既婚,2,NaN,0,4,3,5,0,1606,0,0,0,0,0,0,0,0,1,1,0,356,6,18,-97,8539,3,3,0,1704,3,3,0,1314,6,143,-1496,56020,6,19,-222,6812,0,0,0,0,0,0,0,0,6,12,76351
1,357,2,70+,既婚,2,NaN,0,4,3,5,0,1606,0,0,0,0,0,0,0,0,1,1,0,356,6,18,-97,8539,3,3,0,1704,3,3,0,1314,6,143,-1496,56020,6,19,-222,6812,0,0,0,0,0,0,0,0,6,12,76351
2,357,3,70+,既婚,2,NaN,0,4,3,5,0,1606,0,0,0,0,0,0,0,0,1,1,0,356,6,18,-97,8539,3,3,0,1704,3,3,0,1314,6,143,-1496,56020,6,19,-222,6812,0,0,0,0,0,0,0,0,6,12,76351
3,357,4,70+,既婚,2,NaN,0,4,3,5,0,1606,0,0,0,0,0,0,0,0,1,1,0,356,6,18,-97,8539,3,3,0,1704,3,3,0,1314,6,143,-1496,56020,6,19,-222,6812,0,0,0,0,0,0,0,0,6,12,76351
4,357,5,70+,既婚,2,NaN,0,4,3,5,0,1606,0,0,0,0,0,0,0,0,1,1,0,356,6,18,-97,8539,3,3,0,1704,3,3,0,1314,6,143,-1496,56020,6,19,-222,6812,0,0,0,0,0,0,0,0,6,12,76351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3911,712,7,50-59,NaN,3,1,0,9,2,2,0,1207,0,0,0,0,0,0,0,0,3,7,0,2840,1,1,0,1066,2,2,0,708,0,0,0,0,3,40,0,14513,1,2,0,211,0,0,0,0,0,0,0,0,3,54,20545
3912,712,8,50-59,NaN,3,1,0,9,2,2,0,1207,0,0,0,0,0,0,0,0,3,7,0,2840,1,1,0,1066,2,2,0,708,0,0,0,0,3,40,0,14513,1,2,0,211,0,0,0,0,0,0,0,0,3,54,20545
3913,712,9,50-59,NaN,3,1,0,9,2,2,0,1207,0,0,0,0,0,0,0,0,3,7,0,2840,1,1,0,1066,2,2,0,708,0,0,0,0,3,40,0,14513,1,2,0,211,0,0,0,0,0,0,0,0,3,54,20545
3914,712,10,50-59,NaN,3,1,0,9,2,2,0,1207,0,0,0,0,0,0,0,0,3,7,0,2840,1,1,0,1066,2,2,0,708,0,0,0,0,3,40,0,14513,1,2,0,211,0,0,0,0,0,0,0,0,3,54,20545


## データの前処理

In [59]:
# 欠損値補完前の学習データ
merge_train.head()

,顧客ID,クーポンID,クーポン利用,年齢層,既婚・独身,家族人数,子供人数,持家,年収レベル,c1_購入回数,c1_購入商品数,c1_クーポン割引,c1_購入金額,c2_購入回数,c2_購入商品数,c2_クーポン割引,c2_購入金額,c3_購入回数,c3_購入商品数,c3_クーポン割引,c3_購入金額,c4_購入回数,c4_購入商品数,c4_クーポン割引,c4_購入金額,c5_購入回数,c5_購入商品数,c5_クーポン割引,c5_購入金額,c6_購入回数,c6_購入商品数,c6_クーポン割引,c6_購入金額,c7_購入回数,c7_購入商品数,c7_クーポン割引,c7_購入金額,c8_購入回数,c8_購入商品数,c8_クーポン割引,c8_購入金額,c9_購入回数,c9_購入商品数,c9_クーポン割引,c9_購入金額,c10_購入回数,c10_購入商品数,c10_クーポン割引,c10_購入金額,c11_購入回数,c11_購入商品数,c11_クーポン割引,c11_購入金額,来店頻度,経過日数,総購入金額
0,1,1,0,40-49,NaN,1,NaN,0,5,1,2,0,534,1,2,0,2803,0,0,0,0,1,1,0,764,9,17,0,4981,0,0,0,0,0,0,0,0,9,77,0,22620,9,10,0,24860,0,0,0,0,0,0,0,0,9,19,56562
1,1,2,0,40-49,NaN,1,NaN,0,5,1,2,0,534,1,2,0,2803,0,0,0,0,1,1,0,764,9,17,0,4981,0,0,0,0,0,0,0,0,9,77,0,22620,9,10,0,24860,0,0,0,0,0,0,0,0,9,19,56562
2,1,3,0,40-49,NaN,1,NaN,0,5,1,2,0,534,1,2,0,2803,0,0,0,0,1,1,0,764,9,17,0,4981,0,0,0,0,0,0,0,0,9,77,0,22620,9,10,0,24860,0,0,0,0,0,0,0,0,9,19,56562
3,1,4,0,40-49,NaN,1,NaN,0,5,1,2,0,534,1,2,0,2803,0,0,0,0,1,1,0,764,9,17,0,4981,0,0,0,0,0,0,0,0,9,77,0,22620,9,10,0,24860,0,0,0,0,0,0,0,0,9,19,56562
4,1,5,0,40-49,NaN,1,NaN,0,5,1,2,0,534,1,2,0,2803,0,0,0,0,1,1,0,764,9,17,0,4981,0,0,0,0,0,0,0,0,9,77,0,22620,9,10,0,24860,0,0,0,0,0,0,0,0,9,19,56562


In [145]:
# 欠損値を処理する関数（例）
# mode()[0]は最頻値で補完している
def fill_cust(_dataframe):
    ''' データの欠損部分を、同じカラムの最頻値で埋める '''
    _df = _dataframe.copy()
    
    _df['年齢層'].fillna(_df['年齢層'].mode()[0], inplace=True)
    _df['家族人数'].fillna(_df['家族人数'].mode()[0], inplace=True)
    _df['家族人数'].replace('5+', '5', inplace=True)
    _df['家族人数']
    _df.loc[_df['家族人数'] >= '2' , '既婚・独身'] = _df['既婚・独身'].fillna('既婚')
    _df.loc[_df['家族人数'] < '2' , '既婚・独身'] = _df['既婚・独身'].fillna('独身')
    _df.loc[(_df['家族人数'] >= '2') & (_df['既婚・独身'] == '既婚'), '子供人数'] = _df['子供人数'].fillna(_df['家族人数'].astype(int) - 2)
    _df.loc[(_df['家族人数'] >= '2') & (_df['既婚・独身'] == '独身'), '子供人数'] = _df['子供人数'].fillna(_df['家族人数'].astype(int) - 1)
    _df.loc[_df['家族人数'] < '2', '子供人数'] = _df['子供人数'].fillna(value=0)
    _df['子供人数'].fillna(_df['子供人数'].mode()[0], inplace=True)
    _df['持家'].fillna(_df['持家'].mode()[0], inplace=True)
    _df["年収レベル"].fillna(_df["年収レベル"].mode()[0], inplace=True)
    _df['経過日数'].fillna(_df['経過日数'].mode()[0], inplace=True)
    return _df

In [146]:
merge_train['家族人数']

0       1
1       1
2       1
3       1
4       1
       ..
3911    2
3912    2
3913    2
3914    2
3915    2
Name: 家族人数, Length: 3916, dtype: object

In [147]:
# 学習データの欠損値処理
filled_train = fill_cust(merge_train)
filled_test = fill_cust(merge_test)

In [148]:
# 確認
filled_train.head()

,顧客ID,クーポンID,クーポン利用,年齢層,既婚・独身,家族人数,子供人数,持家,年収レベル,c1_購入回数,c1_購入商品数,c1_クーポン割引,c1_購入金額,c2_購入回数,c2_購入商品数,c2_クーポン割引,c2_購入金額,c3_購入回数,c3_購入商品数,c3_クーポン割引,c3_購入金額,c4_購入回数,c4_購入商品数,c4_クーポン割引,c4_購入金額,c5_購入回数,c5_購入商品数,c5_クーポン割引,c5_購入金額,c6_購入回数,c6_購入商品数,c6_クーポン割引,c6_購入金額,c7_購入回数,c7_購入商品数,c7_クーポン割引,c7_購入金額,c8_購入回数,c8_購入商品数,c8_クーポン割引,c8_購入金額,c9_購入回数,c9_購入商品数,c9_クーポン割引,c9_購入金額,c10_購入回数,c10_購入商品数,c10_クーポン割引,c10_購入金額,c11_購入回数,c11_購入商品数,c11_クーポン割引,c11_購入金額,来店頻度,経過日数,総購入金額
0,1,1,0,40-49,独身,1,0,0,5,1,2,0,534,1,2,0,2803,0,0,0,0,1,1,0,764,9,17,0,4981,0,0,0,0,0,0,0,0,9,77,0,22620,9,10,0,24860,0,0,0,0,0,0,0,0,9,19,56562
1,1,2,0,40-49,独身,1,0,0,5,1,2,0,534,1,2,0,2803,0,0,0,0,1,1,0,764,9,17,0,4981,0,0,0,0,0,0,0,0,9,77,0,22620,9,10,0,24860,0,0,0,0,0,0,0,0,9,19,56562
2,1,3,0,40-49,独身,1,0,0,5,1,2,0,534,1,2,0,2803,0,0,0,0,1,1,0,764,9,17,0,4981,0,0,0,0,0,0,0,0,9,77,0,22620,9,10,0,24860,0,0,0,0,0,0,0,0,9,19,56562
3,1,4,0,40-49,独身,1,0,0,5,1,2,0,534,1,2,0,2803,0,0,0,0,1,1,0,764,9,17,0,4981,0,0,0,0,0,0,0,0,9,77,0,22620,9,10,0,24860,0,0,0,0,0,0,0,0,9,19,56562
4,1,5,0,40-49,独身,1,0,0,5,1,2,0,534,1,2,0,2803,0,0,0,0,1,1,0,764,9,17,0,4981,0,0,0,0,0,0,0,0,9,77,0,22620,9,10,0,24860,0,0,0,0,0,0,0,0,9,19,56562


In [149]:
filled_test.head()

,顧客ID,クーポンID,年齢層,既婚・独身,家族人数,子供人数,持家,年収レベル,c1_購入回数,c1_購入商品数,c1_クーポン割引,c1_購入金額,c2_購入回数,c2_購入商品数,c2_クーポン割引,c2_購入金額,c3_購入回数,c3_購入商品数,c3_クーポン割引,c3_購入金額,c4_購入回数,c4_購入商品数,c4_クーポン割引,c4_購入金額,c5_購入回数,c5_購入商品数,c5_クーポン割引,c5_購入金額,c6_購入回数,c6_購入商品数,c6_クーポン割引,c6_購入金額,c7_購入回数,c7_購入商品数,c7_クーポン割引,c7_購入金額,c8_購入回数,c8_購入商品数,c8_クーポン割引,c8_購入金額,c9_購入回数,c9_購入商品数,c9_クーポン割引,c9_購入金額,c10_購入回数,c10_購入商品数,c10_クーポン割引,c10_購入金額,c11_購入回数,c11_購入商品数,c11_クーポン割引,c11_購入金額,来店頻度,経過日数,総購入金額
0,357,1,70+,既婚,2,0,0,4,3,5,0,1606,0,0,0,0,0,0,0,0,1,1,0,356,6,18,-97,8539,3,3,0,1704,3,3,0,1314,6,143,-1496,56020,6,19,-222,6812,0,0,0,0,0,0,0,0,6,12,76351
1,357,2,70+,既婚,2,0,0,4,3,5,0,1606,0,0,0,0,0,0,0,0,1,1,0,356,6,18,-97,8539,3,3,0,1704,3,3,0,1314,6,143,-1496,56020,6,19,-222,6812,0,0,0,0,0,0,0,0,6,12,76351
2,357,3,70+,既婚,2,0,0,4,3,5,0,1606,0,0,0,0,0,0,0,0,1,1,0,356,6,18,-97,8539,3,3,0,1704,3,3,0,1314,6,143,-1496,56020,6,19,-222,6812,0,0,0,0,0,0,0,0,6,12,76351
3,357,4,70+,既婚,2,0,0,4,3,5,0,1606,0,0,0,0,0,0,0,0,1,1,0,356,6,18,-97,8539,3,3,0,1704,3,3,0,1314,6,143,-1496,56020,6,19,-222,6812,0,0,0,0,0,0,0,0,6,12,76351
4,357,5,70+,既婚,2,0,0,4,3,5,0,1606,0,0,0,0,0,0,0,0,1,1,0,356,6,18,-97,8539,3,3,0,1704,3,3,0,1314,6,143,-1496,56020,6,19,-222,6812,0,0,0,0,0,0,0,0,6,12,76351


In [162]:
# カテゴリ変数のダミー変数化
# 不要なカラムも削除する。
dum_train = pd.get_dummies(filled_train.drop(["顧客ID", "クーポンID", "クーポン利用","持家","既婚・独身"], axis=1))
dum_test = pd.get_dummies(filled_test.drop(["顧客ID", "クーポンID","持家","既婚・独身"], axis=1))

In [163]:
dum_train.head()

,年収レベル,c1_購入回数,c1_購入商品数,c1_クーポン割引,c1_購入金額,c2_購入回数,c2_購入商品数,c2_クーポン割引,c2_購入金額,c3_購入回数,c3_購入商品数,c3_クーポン割引,c3_購入金額,c4_購入回数,c4_購入商品数,c4_クーポン割引,c4_購入金額,c5_購入回数,c5_購入商品数,c5_クーポン割引,c5_購入金額,c6_購入回数,c6_購入商品数,c6_クーポン割引,c6_購入金額,c7_購入回数,c7_購入商品数,c7_クーポン割引,c7_購入金額,c8_購入回数,c8_購入商品数,c8_クーポン割引,c8_購入金額,c9_購入回数,c9_購入商品数,c9_クーポン割引,c9_購入金額,c10_購入回数,c10_購入商品数,c10_クーポン割引,c10_購入金額,c11_購入回数,c11_購入商品数,c11_クーポン割引,c11_購入金額,来店頻度,経過日数,総購入金額,年齢層_20-29,年齢層_30-39,年齢層_40-49,年齢層_50-59,年齢層_60-69,年齢層_70+,家族人数_1,家族人数_2,家族人数_3,家族人数_4,家族人数_5,子供人数_0,子供人数_1,子供人数_1,子供人数_2,子供人数_3+
0,5,1,2,0,534,1,2,0,2803,0,0,0,0,1,1,0,764,9,17,0,4981,0,0,0,0,0,0,0,0,9,77,0,22620,9,10,0,24860,0,0,0,0,0,0,0,0,9,19,56562,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0
1,5,1,2,0,534,1,2,0,2803,0,0,0,0,1,1,0,764,9,17,0,4981,0,0,0,0,0,0,0,0,9,77,0,22620,9,10,0,24860,0,0,0,0,0,0,0,0,9,19,56562,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0
2,5,1,2,0,534,1,2,0,2803,0,0,0,0,1,1,0,764,9,17,0,4981,0,0,0,0,0,0,0,0,9,77,0,22620,9,10,0,24860,0,0,0,0,0,0,0,0,9,19,56562,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0
3,5,1,2,0,534,1,2,0,2803,0,0,0,0,1,1,0,764,9,17,0,4981,0,0,0,0,0,0,0,0,9,77,0,22620,9,10,0,24860,0,0,0,0,0,0,0,0,9,19,56562,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0
4,5,1,2,0,534,1,2,0,2803,0,0,0,0,1,1,0,764,9,17,0,4981,0,0,0,0,0,0,0,0,9,77,0,22620,9,10,0,24860,0,0,0,0,0,0,0,0,9,19,56562,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0


In [164]:
dum_test.head()

,年収レベル,c1_購入回数,c1_購入商品数,c1_クーポン割引,c1_購入金額,c2_購入回数,c2_購入商品数,c2_クーポン割引,c2_購入金額,c3_購入回数,c3_購入商品数,c3_クーポン割引,c3_購入金額,c4_購入回数,c4_購入商品数,c4_クーポン割引,c4_購入金額,c5_購入回数,c5_購入商品数,c5_クーポン割引,c5_購入金額,c6_購入回数,c6_購入商品数,c6_クーポン割引,c6_購入金額,c7_購入回数,c7_購入商品数,c7_クーポン割引,c7_購入金額,c8_購入回数,c8_購入商品数,c8_クーポン割引,c8_購入金額,c9_購入回数,c9_購入商品数,c9_クーポン割引,c9_購入金額,c10_購入回数,c10_購入商品数,c10_クーポン割引,c10_購入金額,c11_購入回数,c11_購入商品数,c11_クーポン割引,c11_購入金額,来店頻度,経過日数,総購入金額,年齢層_20-29,年齢層_30-39,年齢層_40-49,年齢層_50-59,年齢層_60-69,年齢層_70+,家族人数_1,家族人数_2,家族人数_3,家族人数_4,家族人数_5,子供人数_0,子供人数_1,子供人数_1,子供人数_2,子供人数_3+
0,4,3,5,0,1606,0,0,0,0,0,0,0,0,1,1,0,356,6,18,-97,8539,3,3,0,1704,3,3,0,1314,6,143,-1496,56020,6,19,-222,6812,0,0,0,0,0,0,0,0,6,12,76351,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0
1,4,3,5,0,1606,0,0,0,0,0,0,0,0,1,1,0,356,6,18,-97,8539,3,3,0,1704,3,3,0,1314,6,143,-1496,56020,6,19,-222,6812,0,0,0,0,0,0,0,0,6,12,76351,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0
2,4,3,5,0,1606,0,0,0,0,0,0,0,0,1,1,0,356,6,18,-97,8539,3,3,0,1704,3,3,0,1314,6,143,-1496,56020,6,19,-222,6812,0,0,0,0,0,0,0,0,6,12,76351,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0
3,4,3,5,0,1606,0,0,0,0,0,0,0,0,1,1,0,356,6,18,-97,8539,3,3,0,1704,3,3,0,1314,6,143,-1496,56020,6,19,-222,6812,0,0,0,0,0,0,0,0,6,12,76351,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0
4,4,3,5,0,1606,0,0,0,0,0,0,0,0,1,1,0,356,6,18,-97,8539,3,3,0,1704,3,3,0,1314,6,143,-1496,56020,6,19,-222,6812,0,0,0,0,0,0,0,0,6,12,76351,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0


In [165]:
# 学習用の説明変数（X_train）、学習用の目的変数（y_train）、評価用の説明変数（X_test）の用意
X_train = dum_train.copy()
y_train = merge_train["クーポン利用"]
X_test = dum_test.copy()

# モデリング

In [166]:
# モデルインスタンスの生成（RandomForests）
mod = RandomForestClassifier(max_depth=3, n_estimators=500)

# モデルの学習
mod.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=3, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [167]:
# 学習済みのモデルを使用してテストデータに関する予測値（予測確率）を算出する
yproba = mod.predict_proba(X_test)[:, 1]

In [168]:
yproba

array([0.00561279, 0.00561279, 0.00561279, ..., 0.00452486, 0.00452486,
       0.00452486])

# 提出用ファイルの作成

In [169]:
# 顧客ID/クーポンIDごとに予測値（予測確率）の値を並べる
submit = merge_test[["顧客ID", "クーポンID"]].copy()
submit["yproba"] = yproba

In [170]:
submit

,顧客ID,クーポンID,yproba
0,357,1,0.005613
1,357,2,0.005613
2,357,3,0.005613
3,357,4,0.005613
4,357,5,0.005613
...,...,...,...
3911,712,7,0.004525
3912,712,8,0.004525
3913,712,9,0.004525
3914,712,10,0.004525


In [171]:
# 顧客ID × クーポンID（11列）の表になるように並び替える
submit_pivot = submit.pivot(index='顧客ID', columns='クーポンID', values='yproba')

In [172]:
submit_pivot

クーポンID,1,2,3,4,5,6,7,8,9,10,11
顧客ID,,,,,,,,,,,
357,0.005613,0.005613,0.005613,0.005613,0.005613,0.005613,0.005613,0.005613,0.005613,0.005613,0.005613
358,0.005528,0.005528,0.005528,0.005528,0.005528,0.005528,0.005528,0.005528,0.005528,0.005528,0.005528
359,0.046233,0.046233,0.046233,0.046233,0.046233,0.046233,0.046233,0.046233,0.046233,0.046233,0.046233
360,0.011172,0.011172,0.011172,0.011172,0.011172,0.011172,0.011172,0.011172,0.011172,0.011172,0.011172
361,0.015558,0.015558,0.015558,0.015558,0.015558,0.015558,0.015558,0.015558,0.015558,0.015558,0.015558
...,...,...,...,...,...,...,...,...,...,...,...
708,0.006848,0.006848,0.006848,0.006848,0.006848,0.006848,0.006848,0.006848,0.006848,0.006848,0.006848
709,0.043930,0.043930,0.043930,0.043930,0.043930,0.043930,0.043930,0.043930,0.043930,0.043930,0.043930
710,0.003737,0.003737,0.003737,0.003737,0.003737,0.003737,0.003737,0.003737,0.003737,0.003737,0.003737


In [173]:
# 予測ファイルの生成
submit_pivot.to_csv("/content/drive/MyDrive/Colab Notebooks/data/my_submission.csv", header=None)